<a href="https://colab.research.google.com/github/nukano0522/pytorch/blob/master/bert_ner/torch_ner_bert%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.21.2 fugashi==1.1.2 ipadic==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.8 MB/s 
     |████████████████████████████████| 568 kB 73.5 MB/s 
     |████████████████████████████████| 13.4 MB 36.3 MB/s 
     |████████████████████████████████| 120 kB 76.2 MB/s 
     |████████████████████████████████| 6.6 MB 6.4 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=7a8e19f38e7474e8d00548ba79c6123233726dc6531f0468499972dccd65d3d6
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [2]:
import os
import json
import unicodedata
import itertools
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
from typing import List, Optional, Tuple, Union
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification, BertModel
from transformers.modeling_outputs import TokenClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers import BertConfig, BertModel

config = BertConfig()

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [3]:
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


torch_fix_seed()

In [4]:
config.num_labels = 9
print(config.num_labels)
print(config.use_return_dict)
config

9
True


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [5]:
# データのダウンロード
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (32/32), done.
Note: checking out 'f7ed83626d90e5a79f1af99775e4b8c6cba15295'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [6]:
# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json','r'))

# 固有表現のタイプとIDを対応付る辞書 
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

# データセットの分割
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

print(f"Length of train: {len(dataset_train)}")
print(f"Length of val: {len(dataset_val)}")
print(f"Length of test: {len(dataset_test)}")

Length of train: 3205
Length of val: 1068
Length of test: 1070


In [81]:
class NER_tokenizer(BertJapaneseTokenizer):

    def create_tokens_and_labels(self, splitted):
        """分割された文字列をトークン化し、ラベルを付与
        Args：
          splitted: 分割された文字列
            例：
            [{'text': 'レッドフォックス株式会社', 'label': 2},
             {'text': 'は、', 'label': 0},
             {'text': '東京都千代田区', 'label': 5},
             {'text': 'に本社を置くITサービス企業である。', 'label': 0}]
        Return:
          tokens, labels
            例：
            ['レッド', 'フォックス', '株式会社', 'は', '、', '東京', '都', '千代田', '区', 'に', '本社', 'を', '置く', 'IT', 'サービス', '企業', 'で', 'ある', '。']
            [2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        """
        tokens = [] # トークン格納用
        labels = [] # トークンに対応するラベル格納用
        for s in splitted:
            text = s['text']
            label = s['label']
            tokens_splitted = self.tokenize(text) # BertJapaneseTokenizerのトークナイザを使ってトークンに分割
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)
        
        return tokens, labels


    def encoding_for_bert(self, tokens, labels):
        """符号化を行いBERTに入力できる形式にする
        Args:
          tokens: トークン列
          labels: トークンに対応するラベルの列
        Returns: 
          encoding: BERTに入力できる形式
          例：
          {'input_ids': [2, 3990, 13779, 1275, 9, 6, 391, 409, 9674, 280, 7, 2557, 11, 3045, 8267, 1645, 1189, 12, 31, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
          　'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
          　'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
            'labels': [0, 2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

        """
        encoding = self.encode_plus(
            tokens, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        ) 
        # トークン[CLS]、[SEP]のラベルを0
        labels = [0] + labels[:max_length-2] + [0] 
        # トークン[PAD]のラベルを0
        labels = labels + [0]*( max_length - len(labels) ) 
        encoding['labels'] = labels

        return encoding


    def encode_plus_tagged(self, text, entities, max_length):
        """文章とそれに含まれる固有表現が与えられた時に、符号化とラベル列の作成
        Args:
          text: 元の文章
          entities: 文章中の固有表現の位置(span)とラベル(type_id)の情報

        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        entities = sorted(entities, key=lambda x: x['span'][0]) # 固有表現の位置の昇順でソート
        splitted = [] # 分割後の文字列格納用
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # 固有表現ではないものには0のラベルを付与
            splitted.append({'text': text[position:start], 'label':0}) 
            # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
            splitted.append({'text': text[start:end], 'label':label}) 
            position = end

        # 最後の固有表現から文末に、0のラベルを付与
        splitted.append({'text': text[position:], 'label':0})
        # positionとspan[0]の値が同じだと空白文字にラベル0が付与されるため、長さ0の文字列は除く（例：{'text': '', 'label': 0}）
        splitted = [ s for s in splitted if s['text'] ] 

        # 分割された文字列をトークン化し、ラベルを付与
        tokens, labels = self.create_tokens_and_labels(splitted)

        # 符号化を行いBERTに入力できる形式にする
        encoding = self.encoding_for_bert(tokens, labels)

        return encoding


    def encode_plus_untagged(self, text, max_length=None, return_tensors=None):
        """文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークン格納用
        tokens_original = [] # トークンに対応する文章中の文字列格納用
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens) 
        encoding = self.encode_plus(
            tokens, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False,
            return_tensors='pt'
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        return encoding, spans


    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [75]:
import pprint
data = dataset_test[2]
pprint.pprint(data)
text = data["text"]

tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

{'curid': '78589',
 'entities': [{'name': '九州朝日放送', 'span': [16, 22], 'type_id': 2},
              {'name': 'RKB毎日放送', 'span': [23, 30], 'type_id': 2},
              {'name': '福岡放送', 'span': [31, 35], 'type_id': 2},
              {'name': 'TVQ九州放送', 'span': [36, 43], 'type_id': 2},
              {'name': '福岡県', 'span': [54, 57], 'type_id': 5},
              {'name': '福岡', 'span': [82, 84], 'type_id': 5},
              {'name': '佐賀', 'span': [85, 87], 'type_id': 5},
              {'name': '北部九州', 'span': [110, 114], 'type_id': 5},
              {'name': '九州北部地方', 'span': [119, 125], 'type_id': 5}],
 'text': 'また、在福の民放テレビ局のうち、九州朝日放送・RKB毎日放送・福岡放送・TVQ九州放送は、法律上の放送区域は福岡県域であるものの、実際の視聴可能エリアと取材エリアは福岡・佐賀の両県にまたがるため、地域ニュースを扱う時に「北部九州」或いは「九州北部地方」という言葉を使うことが多い。'}


In [76]:
max_length = 32
tokens = [] # トークン格納用
tokens_original = [] # トークンに対応する文章中の文字列格納用
words = tokenizer.word_tokenizer.tokenize(text) # MeCabで単語に分割
# words = tokenizer.tokenize(text) # Bertのトークナイザーで分割

print(words)

for word in words:
    # 単語をサブワードに分割
    tokens_word = tokenizer.subword_tokenizer.tokenize(word) 
    tokens.extend(tokens_word)
    if tokens_word[0] == '[UNK]': # 未知語への対応
        tokens_original.append(word)
    else:
        tokens_original.extend([
            token.replace('##','') for token in tokens_word
        ])

print(tokens)
print(tokens_original)

# 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
position = 0
spans = [] # トークンの位置を追加していく。
for token in tokens_original:
    l = len(token)
    while 1:
        if token != text[position:position+l]:
            print("aaa")
            position += 1
        else:
            spans.append([position, position+l])
            position += l
            break

print(spans)


# 符号化を行いBERTに入力できる形式にする。
input_ids = tokenizer.convert_tokens_to_ids(tokens) 
encoding = tokenizer.encode_plus(
    tokens, 
    max_length=max_length, 
    padding='max_length' if max_length else False, 
    truncation=True if max_length else False
)

print(encoding)




['また', '、', '在', '福', 'の', '民放', 'テレビ局', 'の', 'うち', '、', '九州朝日放送', '・', 'RKB', '毎日放送', '・', '福岡放送', '・', 'TVQ', '九州', '放送', 'は', '、', '法律', '上', 'の', '放送', '区域', 'は', '福岡', '県', '域', 'で', 'ある', 'ものの', '、', '実際', 'の', '視聴', '可能', 'エリア', 'と', '取材', 'エリア', 'は', '福岡', '・', '佐賀', 'の', '両', '県', 'に', 'またがる', 'ため', '、', '地域', 'ニュース', 'を', '扱う', '時', 'に', '「', '北部', '九州', '」', '或いは', '「', '九州', '北部', '地方', '」', 'という', '言葉', 'を', '使う', 'こと', 'が', '多い', '。']
['また', '、', '在', '福', 'の', '民放', 'テレビ局', 'の', 'うち', '、', '九州', '##朝日放送', '・', 'RKB', '毎日放送', '・', '福岡', '##放送', '・', 'TV', '##Q', '九州', '放送', 'は', '、', '法律', '上', 'の', '放送', '区域', 'は', '福岡', '県', '域', 'で', 'ある', 'ものの', '、', '実際', 'の', '視聴', '可能', 'エリア', 'と', '取材', 'エリア', 'は', '福岡', '・', '佐賀', 'の', '両', '県', 'に', 'またがる', 'ため', '、', '地域', 'ニュース', 'を', '扱う', '時', 'に', '「', '北部', '九州', '」', '或いは', '「', '九州', '北部', '地方', '」', 'という', '言葉', 'を', '使う', 'こと', 'が', '多い', '。']
['また', '、', '在', '福', 'の', '民放', 'テレビ局', 'の', 'うち', '、', '九州', '朝日放送', '・', 

In [83]:
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [11]:
import pprint
tmp = dataset_train[1]
pprint.pprint(tmp)
pprint.pprint(tokenizer.encode_plus_tagged(text=tmp["text"], entities=tmp["entities"], max_length=32), width=200)

{'curid': '2415078',
 'entities': [{'name': 'レッドフォックス株式会社', 'span': [0, 12], 'type_id': 2},
              {'name': '東京都千代田区', 'span': [14, 21], 'type_id': 5}],
 'text': 'レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。'}
{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [2, 3990, 13779, 1275, 9, 6, 391, 409, 9674, 280, 7, 2557, 11, 3045, 8267, 1645, 1189, 12, 31, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'labels': [0, 2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [12]:
class CreateDataset(Dataset):
  """データセット作成
  """
  def __init__(self, dataset, tokenizer, max_length):
    self.dataset = dataset
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    text = self.dataset[index]["text"]
    entities = self.dataset[index]["entities"]
    encoding = tokenizer.encode_plus_tagged(text, entities, max_length=self.max_length)

    input_ids = torch.tensor(encoding["input_ids"])
    token_type_ids = torch.tensor(encoding["token_type_ids"])
    attention_mask = torch.tensor(encoding["attention_mask"])
    labels = torch.tensor(encoding["labels"])

    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_mask,
      "labels": labels
    }

In [13]:
# データセットの作成
dataset_train_for_loader = CreateDataset(dataset_train, tokenizer, max_length=128)
dataset_val_for_loader = CreateDataset(dataset_val, tokenizer, max_length=128)

# データローダーの作成
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=32, shuffle=True, pin_memory=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256, shuffle=True, pin_memory=True)

dataloaders_dict = {"train": dataloader_train, "val": dataloader_val}

In [15]:
from torch import nn

model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", add_pooling_layer=False, output_attentions=True, output_hidden_states=True)

class MyBertForTokenClassification(nn.Module):

    def __init__(self):
        super(MyBertForTokenClassification, self).__init__()
        self.num_labels = config.num_labels

        self.bert = model

        self.lstm = torch.nn.LSTM(input_size=config.hidden_size, hidden_size =config.hidden_size, batch_first=True, bidirectional=True)

        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
        self.classifier= nn.Linear(config.hidden_size*2, config.num_labels)

        # 重み初期化処理
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.normal_(self.classifier.bias, 0)


    def forward(
        self,
        input_ids,
        token_type_ids,
        attention_mask,
        labels: Optional[torch.Tensor] = None,
        return_dict: Optional[bool] = None,
        ):
      
        return_dict = return_dict if return_dict is not None else config.use_return_dict

        outputs = self.bert(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            # position_ids=position_ids,
            # head_mask=head_mask,
            # inputs_embeds=inputs_embeds,
            # output_attentions=output_attentions,
            # output_hidden_states=output_hidden_states,
            # labels=labels,
            return_dict=return_dict,
        )
        # print(outputs.keys())

        # lstmout, _ = self.lstm(outputs[0], None)
        lstmout, _ = self.lstm(outputs["last_hidden_state"], None)

        logits = self.classifier(self.dropout(lstmout))

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# GPU使えるならGPU使う
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 学習済みモデルのロード
# model_ = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=9)
model = MyBertForTokenClassification()

model.to(device)

MyBertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [17]:
# 最適化器
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)

# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    output = net(input_ids=input_ids, 
                                          token_type_ids=None, 
                                          attention_mask=attention_mask, 
                                          labels=labels,
                                          return_dict=True)
                    
                    loss = output[0]

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            print(f"イテレーション {iteration} || Loss: {loss:.4f}")

                    iteration += 1

                    # 損失の合計を更新
                    epoch_loss += loss.item() * batch_size

            # epochごとのloss
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print(f"Epoch {epoch+1}/{num_epochs} | phase {phase} |  Loss: {epoch_loss:.4f}")

    return net

In [18]:
# 学習・検証を実行
num_epochs = 5
net_trained = train_model(model, dataloaders_dict, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 1.1337
イテレーション 20 || Loss: 0.8798
イテレーション 30 || Loss: 0.8016
イテレーション 40 || Loss: 0.5322
イテレーション 50 || Loss: 0.3890
イテレーション 60 || Loss: 0.3726
イテレーション 70 || Loss: 0.2967
イテレーション 80 || Loss: 0.3514
イテレーション 90 || Loss: 0.3020
イテレーション 100 || Loss: 0.3128
Epoch 1/5 | phase train |  Loss: 0.6079
Epoch 1/5 | phase val |  Loss: 0.0280
イテレーション 10 || Loss: 0.1672
イテレーション 20 || Loss: 0.1204
イテレーション 30 || Loss: 0.1100
イテレーション 40 || Loss: 0.1729
イテレーション 50 || Loss: 0.1437
イテレーション 60 || Loss: 0.0998
イテレーション 70 || Loss: 0.1918
イテレーション 80 || Loss: 0.1235
イテレーション 90 || Loss: 0.0870
イテレーション 100 || Loss: 0.0910
Epoch 2/5 | phase train |  Loss: 0.1327
Epoch 2/5 | phase val |  Loss: 0.0153
イテレーション 10 || Loss: 0.0896
イテレーション 20 || Loss: 0.0400
イテレーション 30 || Loss: 0.0675
イテレーション 40 || Loss: 0.0837
イテレーション 50 || Loss: 0.0604
イテレーション 60 || Loss: 0.0566
イテレーション 70 || Loss: 0.0358
イテレーション 80 || Loss: 0.0286
イテレーション 90 || Loss: 0.0684
イテレーション 100 || Loss: 0.028

In [19]:
# モデル保存
# torch.save(net_trained.state_dict(), './model.pth')

In [20]:
# # モデルロード
# model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", add_pooling_layer=False, output_attentions=True, output_hidden_states=True)
# net_trained_ = MyBertForTokenClassification()
# net_trained_.load_state_dict(torch.load('./model.pth'), strict=False)
# net_trained_.to(device)

In [84]:
def predict(text, tokenizer, model):
    """BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() }
    # print(encoding)

    # ラベルの予測値の計算
    with torch.no_grad():
        output = model(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist() 

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

# 固有表現抽出
entities_list = [] # 正解の固有表現
entities_predicted_list = [] # 予測された固有表現
for sample in tqdm(dataset_test):
    text = sample['text']
    entities_predicted = predict(text, tokenizer, net_trained) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

100%|██████████| 1070/1070 [00:10<00:00, 104.55it/s]


In [71]:
i = 2
print("# 正解 #")
print(entities_list[i])
print("# 推論 #")
print(entities_predicted_list[i])
print("# もとの文章 #")
print(dataset_test[i]["text"])

# 正解 #
[{'name': '九州朝日放送', 'span': [16, 22], 'type_id': 2}, {'name': 'RKB毎日放送', 'span': [23, 30], 'type_id': 2}, {'name': '福岡放送', 'span': [31, 35], 'type_id': 2}, {'name': 'TVQ九州放送', 'span': [36, 43], 'type_id': 2}, {'name': '福岡県', 'span': [54, 57], 'type_id': 5}, {'name': '福岡', 'span': [82, 84], 'type_id': 5}, {'name': '佐賀', 'span': [85, 87], 'type_id': 5}, {'name': '北部九州', 'span': [110, 114], 'type_id': 5}, {'name': '九州北部地方', 'span': [119, 125], 'type_id': 5}]
# 推論 #
[{'name': '福', 'span': [4, 5], 'type_id': 5}, {'name': '九州朝日放送', 'span': [16, 22], 'type_id': 2}, {'name': 'RKB毎日放送', 'span': [23, 30], 'type_id': 2}, {'name': '福岡放送', 'span': [31, 35], 'type_id': 2}, {'name': 'TVQ九州放送', 'span': [36, 43], 'type_id': 2}, {'name': '福岡県', 'span': [54, 57], 'type_id': 5}, {'name': '福岡', 'span': [82, 84], 'type_id': 5}, {'name': '佐賀', 'span': [85, 87], 'type_id': 5}, {'name': '県', 'span': [89, 90], 'type_id': 5}, {'name': '北部九州', 'span': [110, 114], 'type_id': 5}, {'name': '九州北部地方', 'span': [

In [72]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]
            
        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [73]:
# 評価結果
import pandas as pd
eval_df = pd.DataFrame()
for k, v in type_id_dict.items():
  eval_res = evaluate_model(entities_list, entities_predicted_list, type_id=v)
  eval_df[k] = eval_res.values()

eval_res_all = evaluate_model(entities_list, entities_predicted_list, type_id=None)
eval_df["ALL"] = eval_res_all.values()

eval_df.index = eval_res_all.keys()
eval_df

,人名,法人名,政治的組織名,その他の組織名,地名,施設名,製品名,イベント名,ALL
num_entities,604.000000,504.000000,249.000000,222.000000,452.000000,222.000000,231.000000,203.000000,2687.000000
num_predictions,615.000000,484.000000,259.000000,235.000000,490.000000,235.000000,243.000000,199.000000,2760.000000
num_correct,570.000000,438.000000,210.000000,185.000000,411.000000,182.000000,182.000000,172.000000,2350.000000
precision,0.926829,0.904959,0.810811,0.787234,0.838776,0.774468,0.748971,0.864322,0.851449
recall,0.943709,0.869048,0.843373,0.833333,0.909292,0.819820,0.787879,0.847291,0.874581
f_value,0.935193,0.886640,0.826772,0.809628,0.872611,0.796499,0.767932,0.855721,0.862860
